#### Connecting To SF Session

In [2]:
from snowflake.snowpark import Session
import snowflake.snowpark.types as T
import pandas as pd
import configparser


In [3]:

# Loading Credentials From Config File
snowflake_credentials_file = '../snowflake_creds.config'
config = configparser.ConfigParser()
config.read(snowflake_credentials_file)
connection_parameters = dict(config['default'])

session = Session.builder.configs(connection_parameters).create()

#### Snowflake Object Creations

In [ ]:
session.sql("CREATE WAREHOUSE IF NOT EXISTS COMPUTE_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()
session.sql("CREATE DATABASE IF NOT EXISTS SNOWPARK_DEFINITIVE_GUIDE").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS SNOWPARK_DEFINITIVE_GUIDE.MY_SCHEMA").collect()
session.sql("CREATE STAGE IF NOT EXISTS SNOWPARK_DEFINITIVE_GUIDE.MY_SCHEMA.MY_STAGE").collect()

#### Schema Definition

In [25]:


bsd_schema = T.StructType([
        T.StructField('DATETIME', T.TimestampType()),T.StructField('SEASON', T.IntegerType()),
        T.StructField('HOLIDAY', T.IntegerType()),T.StructField('WORKINGDAY', T.IntegerType()),
        T.StructField('WEATHER', T.IntegerType()),T.StructField('TEMP', T.FloatType()),
        T.StructField('ATEMP', T.FloatType()),T.StructField('HUMIDITY', T.IntegerType()),
        T.StructField('WINDSPEED', T.FloatType()),T.StructField('CASUAL', T.IntegerType()),
        T.StructField('REGISTERED', T.IntegerType()),T.StructField('COUNT', T.IntegerType())
])


#### Read & Transform CSV To Load

In [28]:
bsd_train = pd.read_csv("../datasets/bike_sharing_demand.csv")
bsd_train.columns = ['DATETIME','SEASON','HOLIDAY','WORKINGDAY','WEATHER','TEMP','ATEMP','HUMIDITY','WINDSPEED','CASUAL','REGISTERED','COUNT']
table_name = "BSD_TRAIN"
bsd_train['DATETIME'] = pd.to_datetime(bsd_train['DATETIME'])
bsd_train.head()


,DATETIME,SEASON,HOLIDAY,WORKINGDAY,WEATHER,TEMP,ATEMP,HUMIDITY,WINDSPEED,CASUAL,REGISTERED,COUNT
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


#### Writing It As Snowflake Table

In [29]:
sf_df = session.create_dataframe(bsd_train, schema=bsd_schema)
sf_df.write.mode("overwrite").save_as_table(table_name)
sf_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATETIME"           |"SEASON"  |"HOLIDAY"  |"WORKINGDAY"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"  |"CASUAL"  |"REGISTERED"  |"COUNT"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------
|2011-01-01 00:00:00  |1         |0          |0             |1          |9.84    |14.395   |81          |0.0          |3         |13            |16       |
|2011-01-01 01:00:00  |1         |0          |0             |1          |9.02    |13.635   |80          |0.0          |8         |32            |40       |
|2011-01-01 02:00:00  |1         |0          |0             |1          |9.02    |13.635   |80          |0.0          |5         |27            |32       |
|2011-01-01 03:00:00  |1         |0          |0             |1  